In this notebook, we will go over how to leverage the SDK to directly work interactively with a Ray cluster during development.

In [1]:
# Import pieces from codeflare-sdk
from codeflare_sdk import Cluster, ClusterConfiguration, TokenAuthentication

In [ ]:
# Create authentication object for user permissions
# IF unused, SDK will automatically check for default kubeconfig, then in-cluster config
# KubeConfigFileAuthentication can also be used to specify kubeconfig path manually
auth = TokenAuthentication(
    token = "XXXXX",
    server = "XXXXX",
    skip_tls=False
)
auth.login()

Once again, let's start by running through the same cluster setup as before:

NOTE: We must specify the `image` which will be used in our RayCluster, we recommend you bring your own image which suits your purposes. 
The example here is a community image.

In [2]:
# Create and configure our cluster object
# The SDK will try to find the name of your default local queue based on the annotation "kueue.x-k8s.io/default-queue": "true" unless you specify the local queue manually below
namespace = "default" # Update to your namespace
cluster_name = "interactivetest"
cluster = Cluster(ClusterConfiguration(
    name=cluster_name,
    namespace=namespace,
    head_gpus=1, # For GPU enabled workloads set the head_gpus and num_gpus
    num_gpus=1,
    num_workers=2,
    min_cpus=2,
    max_cpus=2,
    min_memory=8,
    max_memory=8,
    image="quay.io/project-codeflare/ray:latest-py39-cu118",
    write_to_file=False, # When enabled Ray Cluster yaml files are written to /HOME/.codeflare/resources 
    # local_queue="local-queue-name" # Specify the local queue manually
))

Written to: interactivetest.yaml


In [3]:
# Bring up the cluster
cluster.up()
cluster.wait_ready()

Waiting for requested resources to be set up...
Requested cluster up and running!


In [4]:
cluster.details()

                      🚀 CodeFlare Cluster Details 🚀                     
                                                                          
 ╭──────────────────────────────────────────────────────────────────────╮ 
 │   Name                                                               │ 
 │   interactivetest                                        Active ✅   │ 
 │                                                                      │ 
 │   URI: ray://interactivetest-head-svc.default.svc:10001              │ 
 │                                                                      │ 
 │   ]8;id=970589;http://ray-dashboard-interactivetest-default.apps.meyceoz-07122023.psap.aws.rhperfscale.org\Dashboard🔗]8;;\                                                        │ 
 │                                                                      │ 
 │                       Cluster Resources                              │ 
 │   ╭── Workers ──╮  ╭───────── Worker specs(each) ─────────╮          │ 
 │   │  # Workers  │  │  Memory      CPU         GPU         │          │ 
 │   │             │  │                                      │          │ 
 │   │  2          │  │  8~8         2           1           │          │ 
 │   │             │  │                                      │          │ 
 │   ╰─────────────╯  ╰──────────────────────────────────────╯          │ 
 ╰──────────────────────────────────────────────────────────────────────╯

RayCluster(name='interactivetest', status=<RayClusterStatus.READY: 'ready'>, workers=2, worker_mem_min=8, worker_mem_max=8, worker_cpu=2, worker_gpu=1, namespace='default', dashboard='http://ray-dashboard-interactivetest-default.apps.meyceoz-07122023.psap.aws.rhperfscale.org')

This time we will demonstrate another potential method of use: working with the Ray cluster interactively.

Using the SDK, we can get both the Ray cluster URI and dashboard URI:

In [5]:
ray_dashboard_uri = cluster.cluster_dashboard_uri()
ray_cluster_uri = cluster.cluster_uri()
print(ray_dashboard_uri)
print(ray_cluster_uri)

http://ray-dashboard-interactivetest-default.apps.meyceoz-07122023.psap.aws.rhperfscale.org
ray://interactivetest-head-svc.default.svc:10001


Now we can connect directly to our Ray cluster via the Ray python client:

In [ ]:
from codeflare_sdk import generate_cert
# Create required TLS cert and export the environment variables to enable TLS
generate_cert.generate_tls_cert(cluster_name, namespace)
generate_cert.export_env(cluster_name, namespace)

In [6]:
#before proceeding make sure the cluster exists and the uri is not empty
assert ray_cluster_uri, "Ray cluster needs to be started and set before proceeding"

import ray
from ray.air.config import ScalingConfig

# reset the ray context in case there's already one. 
ray.shutdown()
# establish connection to ray cluster

#install additional libraries that will be required for model training
runtime_env = {"pip": ["transformers", "datasets", "evaluate", "pyarrow<7.0.0", "accelerate"]}

# NOTE: This will work for in-cluster notebook servers (RHODS/ODH), but not for local machines
# To see how to connect from your laptop, go to demo-notebooks/additional-demos/local_interactive.ipynb
ray.init(address=ray_cluster_uri, runtime_env=runtime_env)

print("Ray cluster is up and running: ", ray.is_initialized())

Ray cluster is up and running:  True


Now that we are connected (and have passed in some package requirements), let's try writing some training code for a DistilBERT transformer model via HuggingFace (using IMDB dataset):

In [7]:
@ray.remote
def train_fn():
    from datasets import load_dataset
    import transformers
    from transformers import AutoTokenizer, TrainingArguments
    from transformers import AutoModelForSequenceClassification
    import numpy as np
    from datasets import load_metric
    import ray
    from ray import tune
    from ray.train.huggingface import HuggingFaceTrainer

    dataset = load_dataset("imdb")
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    #using a fraction of dataset but you can run with the full dataset
    small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
    small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

    print(f"len of train {small_train_dataset} and test {small_eval_dataset}")

    ray_train_ds = ray.data.from_huggingface(small_train_dataset)
    ray_evaluation_ds = ray.data.from_huggingface(small_eval_dataset)

    def compute_metrics(eval_pred):
        metric = load_metric("accuracy")
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    def trainer_init_per_worker(train_dataset, eval_dataset, **config):
        model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

        training_args = TrainingArguments("/tmp/hf_imdb/test", eval_steps=1, disable_tqdm=True, 
                                          num_train_epochs=1, skip_memory_metrics=True,
                                          learning_rate=2e-5,
                                          per_device_train_batch_size=16,
                                          per_device_eval_batch_size=16,                                
                                          weight_decay=0.01,)
        return transformers.Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            compute_metrics=compute_metrics
        )

    scaling_config = ScalingConfig(num_workers=2, use_gpu=True) #num workers is the number of gpus

    # we are using the ray native HuggingFaceTrainer, but you can swap out to use non ray Huggingface Trainer. Both have the same method signature. 
    # the ray native HFTrainer has built in support for scaling to multiple GPUs
    trainer = HuggingFaceTrainer(
        trainer_init_per_worker=trainer_init_per_worker,
        scaling_config=scaling_config,
        datasets={"train": ray_train_ds, "evaluation": ray_evaluation_ds},
    )
    result = trainer.fit()

Once we want to test our code out, we can run the training function we defined above remotely on our Ray cluster:

In [8]:
#call the above cell as a remote ray function
ray.get(train_fn.remote())

(train_fn pid=425) Downloading and preparing dataset imdb/plain_text to /home/ray/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0...


Generating unsupervised split:  97%|█████████▋| 48664/50000 [00:07<00:00, 10941.00 examples/s]


(train_fn pid=425) Dataset imdb downloaded and prepared to /home/ray/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


Map:  98%|█████████▊| 49000/50000 [00:27<00:00, 1773.03 examples/s]


(train_fn pid=425) len of train Dataset({
(train_fn pid=425)     features: ['text', 'label', 'input_ids', 'attention_mask'],
(train_fn pid=425)     num_rows: 100
(train_fn pid=425) }) and test Dataset({
(train_fn pid=425)     features: ['text', 'label', 'input_ids', 'attention_mask'],
(train_fn pid=425)     num_rows: 100
(train_fn pid=425) })


...

(train_fn pid=425) == Status ==
(train_fn pid=425) Current time: 2023-08-09 14:52:01 (running for 00:00:10.18)
(train_fn pid=425) Using FIFO scheduling algorithm.
(train_fn pid=425) Logical resource usage: 1.0/6 CPUs, 2.0/2 GPUs
(train_fn pid=425) Result logdir: /home/ray/ray_results/HuggingFaceTrainer_2023-08-09_14-51-51
(train_fn pid=425) Number of trials: 1/1 (1 RUNNING)
(train_fn pid=425) +--------------------------------+----------+-----------------+
(train_fn pid=425) | Trial name                     | status   | loc             |
(train_fn pid=425) |--------------------------------+----------+-----------------|
(train_fn pid=425) | HuggingFaceTrainer_f2621_00000 | RUNNING  | 10.130.4.19:196 |
(train_fn pid=425) +--------------------------------+----------+-----------------+
(train_fn pid=425) 
(train_fn pid=425) 


(RayTrainWorker pid=235, ip=10.130.4.19) 2023-08-09 14:52:01,262	INFO streaming_executor.py:91 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[RandomizeBlockOrder]
(RayTrainWorker pid=235, ip=10.130.4.19) 2023-08-09 14:52:01,262	INFO streaming_executor.py:92 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
(RayTrainWorker pid=235, ip=10.130.4.19) 2023-08-09 14:52:01,262	INFO streaming_executor.py:94 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`
(RayTrainWorker pid=235, ip=10.130.4.19) 2023-08-09 14:52:01,274	INFO streaming_executor.py:149 -- Shutting down <StreamingExecutor(Thread-3, stopped daemon 140636397225728)>.
(RayTrainWorker pid=232, ip=10.129.4.19) 2023-08-09 14:52:01,252	INFO streaming_executor.py:91 -- Executing DAG

(train_fn pid=425) == Status ==
(train_fn pid=425) Current time: 2023-08-09 14:52:06 (running for 00:00:15.20)
(train_fn pid=425) Using FIFO scheduling algorithm.
(train_fn pid=425) Logical resource usage: 1.0/6 CPUs, 2.0/2 GPUs
(train_fn pid=425) Result logdir: /home/ray/ray_results/HuggingFaceTrainer_2023-08-09_14-51-51
(train_fn pid=425) Number of trials: 1/1 (1 RUNNING)
(train_fn pid=425) +--------------------------------+----------+-----------------+
(train_fn pid=425) | Trial name                     | status   | loc             |
(train_fn pid=425) |--------------------------------+----------+-----------------|
(train_fn pid=425) | HuggingFaceTrainer_f2621_00000 | RUNNING  | 10.130.4.19:196 |
(train_fn pid=425) +--------------------------------+----------+-----------------+
(train_fn pid=425) 
(train_fn pid=425) 
(train_fn pid=425) == Status ==
(train_fn pid=425) Current time: 2023-08-09 14:52:11 (running for 00:00:20.22)
(train_fn pid=425) Using FIFO scheduling algorithm.
(trai

(train_fn pid=425) 2023-08-09 14:57:48,184	WARNING util.py:315 -- The `process_trial_save` operation took 8.180 s, which may be a performance bottleneck.
(train_fn pid=425) 2023-08-09 14:57:48,185	WARNING trial_runner.py:928 -- Consider turning off forced head-worker trial checkpoint syncs by setting sync_on_checkpoint=False. Note that this may result in faulty trial restoration if a failure occurs while the checkpoint is being synced from the worker to the head node.


(train_fn pid=425) Trial HuggingFaceTrainer_f2621_00000 completed.
(train_fn pid=425) == Status ==
(train_fn pid=425) Current time: 2023-08-09 14:57:50 (running for 00:05:59.21)
(train_fn pid=425) Using FIFO scheduling algorithm.
(train_fn pid=425) Logical resource usage: 1.0/6 CPUs, 2.0/2 GPUs
(train_fn pid=425) Result logdir: /home/ray/ray_results/HuggingFaceTrainer_2023-08-09_14-51-51
(train_fn pid=425) Number of trials: 1/1 (1 TERMINATED)
(train_fn pid=425) +--------------------------------+------------+-----------------+--------+------------------+--------+-----------------+---------+
(train_fn pid=425) | Trial name                     | status     | loc             |   iter |   total time (s) |   loss |   learning_rate |   epoch |
(train_fn pid=425) |--------------------------------+------------+-----------------+--------+------------------+--------+-----------------+---------|
(train_fn pid=425) | HuggingFaceTrainer_f2621_00000 | TERMINATED | 10.130.4.19:196 |      1 |          

Once complete, we can bring our Ray cluster down and clean up:

In [9]:
cluster.down()

In [ ]:
auth.logout()